In [ ]:
#need to get similairty cluster mean scores for all tweets
#takes a while because each tweet needs to be measured 30 times
#create functions that will run in parallel
#this script works for republicans
#the second splits into the 8 months, 16 total processes running
#one the distances are created they are saved to the distances folder

In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from numpy import dot
from numpy.linalg import norm
import multiprocessing as mp

OUTPUT = "/home/twalton_umass_edu/Political Polarization Project/tmls/word_embeddings/clustering/"
INPUT = '/home/twalton_umass_edu/Political Polarization Project/tmls/word_embeddings/sent_embed/'
NUM_CLUSTERS = 40

#create list of numbers to represent months
months = range(8)
#load model
model = Word2Vec.load('/home/twalton_umass_edu/Political Polarization Project/tmls/word_embeddings/rep_word2vec_100.model')
#load means
means = np.load('/home/twalton_umass_edu/Political Polarization Project/tmls/word_embeddings/clustering/rep_cluster_' + str(NUM_CLUSTERS) + '_means.npy')

#cosine similarity function
def cosine_sim(a, b):
    return dot(a, b) / (norm(a)*norm(b))

In [3]:
#function that gets the scores/saves for each month
def get_scores(m):
    
    #load cleaned text dataframe
    text = pd.read_csv(INPUT + 'cleaned/rep_clean_' + str(m) + '.csv')
    #load sentence embeddings
    embedding = pd.read_csv(INPUT + 'trained/rep_trained_' + str(m) + '.csv')
    
    print('length of text: ' + str(len(text)))
    print('length of embedding: ' + str(len(embedding)))
    
    #remove unnamed axxis from embedding
    embedding = embedding.drop(['Unnamed: 0'], axis = 1)
    
    dicts = []
    
    for embed in range(len(embedding)):
        d = {}
        distances = np.array([cosine_sim(embedding.loc[embed], me) for me in means])
        sorted_dists = distances.argsort()
        for i, topic in enumerate(sorted_dists):
            d['topic_' + str(i)] = topic  # ith closest topic
            d['cosine_' + str(i)] = distances[topic]  # cosine distance of ith closest topic
        dicts.append(d)
    df = pd.DataFrame(dicts)
    df.to_csv(OUTPUT + 'distances/rep_kmeans_topics_' + str(NUM_CLUSTERS) + '_' + str(m) + '.csv', index=False)

In [ ]:
pool = mp.Pool(mp.cpu_count())

results = pool.map(get_scores, [m for m in months])

pool.close()

length of text: 2294805
length of embedding: 2294805
length of text: 1230169
length of embedding: 1230169
length of text: 2015834
length of embedding: 2015834
length of text: 2406714
length of embedding: 2406714
length of text: 2750488
length of embedding: 2750488
length of text: 2694886
length of embedding: 2694886
length of text: 2527290
length of embedding: 2527290
length of text: 2464728
length of embedding: 2464728


In [3]:
get_scores(3)

length of text: 2464728
length of embedding: 2464728
